In [19]:
#version 20: full country list cached, visible count under dropdown
import dash
from dash import dcc, html, Input, Output, State
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import wbdata
from datetime import datetime

# Step 2: Prepare the Data
mnch_outcomes = {
    'Gestational Diabetes': {'odds_ratio': 1.07, 'current_rate': 0.05, 'cost_per_case': 2000, 'ci_lower': 1.01, 'ci_upper': 1.54},
    'Hypertension during Pregnancy': {'odds_ratio': 1.12, 'current_rate': 0.1, 'cost_per_case': 1500, 'ci_lower': 1.03, 'ci_upper': 1.21},
    'Maternal Admission': {'odds_ratio': 1.01, 'current_rate': 0.02, 'cost_per_case': 1000, 'ci_lower': 1.00, 'ci_upper': 1.03},
    'Infections': {'odds_ratio': 1.29, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.05, 'ci_upper': 1.58},
    'Prelabour Rupture of Membranes': {'odds_ratio': 1.08, 'current_rate': 0.02, 'cost_per_case': 1200, 'ci_lower': 1.07, 'ci_upper': 1.09},
    'Antenatal Bleeding': {'odds_ratio': 1.16, 'current_rate': 0.005, 'cost_per_case': 2500, 'ci_lower': 1.13, 'ci_upper': 1.40},
    'Cardiovascular Event': {'odds_ratio': 1.11, 'current_rate': 0.01, 'cost_per_case': 3000, 'ci_lower': 1.06, 'ci_upper': 1.15},
    'Stillbirth': {'odds_ratio': 1.14, 'current_rate': 0.005, 'cost_per_case': 5000, 'ci_lower': 0.99, 'ci_upper': 1.32},
    'Congenital Anomalies': {'odds_ratio': 1.13, 'current_rate': 0.02, 'cost_per_case': 10000, 'ci_lower': 0.99, 'ci_upper': 1.85},
    'Spontaneous Abortion': {'odds_ratio': 1.31, 'current_rate': 0.01, 'cost_per_case': 5000, 'ci_lower': 0.99, 'ci_upper': 3.30},
    'Non-reassuring Fetal Status': {'odds_ratio': 1.21, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.12, 'ci_upper': 1.32},
    'Composite Outcome': {'odds_ratio': 1.42, 'current_rate': 0.1, 'cost_per_case': 2000, 'ci_lower': 1.00, 'ci_upper': 2.03},
    'Preterm Birth': {'odds_ratio': 1.04, 'current_rate': 0.1, 'cost_per_case': 32000, 'ci_lower': 1.03, 'ci_upper': 1.06},
    'Low Birth Weight': {'odds_ratio': 1.13, 'current_rate': 0.08, 'cost_per_case': 800, 'ci_lower': 1.02, 'ci_upper': 1.26},
    'Small for Gestational Age': {'odds_ratio': 1.10, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.02, 'ci_upper': 1.18},
    'Neonatal Admission': {'odds_ratio': 1.22, 'current_rate': 0.02, 'cost_per_case': 1500, 'ci_lower': 1.02, 'ci_upper': 1.43},
    'Neonatal Morbidity': {'odds_ratio': 1.04, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.03, 'ci_upper': 1.06},
    'Obstetric Complication': {'odds_ratio': 1.05, 'current_rate': 0.1, 'cost_per_case': 1500, 'ci_lower': 1.03, 'ci_upper': 1.06}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

# Helpers: flags and country options

def iso2_to_flag(iso2: str) -> str:
    try:
        if not iso2 or len(iso2) != 2:
            return "🏳️"
        return ''.join(chr(127397 + ord(c.upper())) for c in iso2)
    except Exception:
        return "🏳️"

def get_country_options():
    try:
        countries = wbdata.get_countries()
        countries_sorted = sorted(countries, key=lambda c: c.get('name', ''))
        opts = []
        for c in countries_sorted:
            iso2 = c.get('iso2Code') or c.get('code') or ''
            flag = iso2_to_flag(iso2)
            label = f"{flag} {c.get('name', '')}"
            opts.append({'label': label, 'value': c.get('id')})  # value uses WB country id
        return opts
    except Exception:
        # Fallback without flags
        return [{'label': f"🏳️ {c['name']}", 'value': c['id']} for c in wbdata.get_countries()]

# Cache full country list once
countries_options = get_country_options()


def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

def calculate_additional_outcomes_and_costs(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    ci_lower = mnch_outcomes[outcome]['ci_lower']
    ci_upper = mnch_outcomes[outcome]['ci_upper']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    # Calculate confidence intervals for effective additional outcomes
    effective_additional_outcomes_lower = [outcome * (1 - adaptation_effectiveness / 100) * ci_lower / odds_ratio for outcome in additional_outcomes]
    effective_additional_outcomes_upper = [outcome * (1 - adaptation_effectiveness / 100) * ci_upper / odds_ratio for outcome in additional_outcomes]
    
    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]
    costs_lower = [outcome * cost_per_case for outcome in effective_additional_outcomes_lower]
    costs_upper = [outcome * cost_per_case for outcome in effective_additional_outcomes_upper]
    
    return selected_years, effective_additional_outcomes, costs, additional_outcomes, effective_additional_outcomes_lower, effective_additional_outcomes_upper, costs_lower, costs_upper

# Calculate Attributable Fraction
def calculate_attributable_fraction(odds_ratio):
    return (odds_ratio - 1) / odds_ratio

# Fetch population data from World Bank
def fetch_population_data(country_code):
    try:
        # Fetch the latest available population data
        print(f"Fetching population data for {country_code}...")
        population_data = wbdata.get_dataframe({'SP.POP.TOTL': 'population'}, country=country_code)
        if population_data.empty:
            print(f"No population data available for {country_code}")
            return None
        population_data = population_data.reset_index()
        population_data['date'] = pd.to_datetime(population_data['date'])
        latest_population = population_data.loc[population_data['date'].idxmax()]
        return latest_population['population']
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Step 3: Create the Dash Layout with Sidebar + Main
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Link(rel='stylesheet', href='/assets/style.css'),
    
    # Header Section
    html.Div(className='header-section', children=[
        html.H1("Climate Health Impact Dashboard"),
        html.H2("Maternal & Child Health Outcomes Due to Rising Temperatures", className='subtitle')
    ]),
    
    # Page Grid: Sidebar + Main
    html.Div(className='page', children=[
        # Sidebar Controls
        html.Div(className='sidebar expanded', id='sidebar', children=[
            html.Button('⟨', id='sidebar-toggle', className='sidebar-toggle', title='Collapse / Expand'),
            html.H2("Configuration", className='section-title hide-when-collapsed'),
            html.Div(className='input-container hide-when-collapsed', children=[
                html.Div(className='input-box', children=[
                    html.H3("🌍 Geographic Scope"),
                    dcc.Dropdown(
                        id='country-dropdown',
                        options=[{'label': '🌍 Global', 'value': 'global'}] + countries_options,
                        value='global',
                        className='dropdown',
                        placeholder="Select geographic scope...",
                        searchable=True,
                        clearable=False
                    ),
                    html.Div(className='helper-text', children=[f"{len(countries_options)} countries loaded"])
                ]),
                html.Div(className='input-box', children=[
                    html.H3("👥 Current Population"),
                    dcc.Input(
                        id='current-population-input',
                        type='number',
                        value=7000000000,
                        className='input',
                        placeholder="Enter population size..."
                    ),
                    dcc.Checklist(
                        id='manual-input-checklist',
                        options=[{'label': ' Manual Input Override', 'value': 'manual'}],
                        value=[],
                        className='checkbox-container'
                    )
                ]),
                html.Div(className='input-box', children=[
                    html.H3("🏥 Health Outcome"),
                    dcc.Dropdown(
                        id='mnch-outcome-dropdown',
                        options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
                        value='Preterm Birth',
                        className='dropdown',
                        placeholder="Select health outcome..."
                    )
                ]),
                html.Div(className='input-box', children=[
                    html.H3("🌡️ Climate Scenario"),
                    dcc.Dropdown(
                        id='rcp-scenario-dropdown',
                        options=[
                            {'label': 'RCP 2.6 (Best Case) - +1.5°C', 'value': 'RCP 2.6'},
                            {'label': 'RCP 4.5 (Moderate) - +2.4°C', 'value': 'RCP 4.5'},
                            {'label': 'RCP 6.0 (High) - +3.0°C', 'value': 'RCP 6.0'},
                            {'label': 'RCP 8.5 (Worst Case) - +4.3°C', 'value': 'RCP 8.5'}
                        ],
                        value='RCP 2.6',
                        className='dropdown',
                        placeholder="Select climate scenario..."
                    )
                ]),
                html.Div(className='input-box', children=[
                    html.H3("📈 Population Growth"),
                    dcc.Dropdown(
                        id='population-growth-dropdown',
                        options=[
                            {'label': '📉 Low Growth (1% annually)', 'value': 'Low Growth'},
                            {'label': '📊 Medium Growth (2% annually)', 'value': 'Medium Growth'},
                            {'label': '📈 High Growth (3% annually)', 'value': 'High Growth'}
                        ],
                        value='Medium Growth',
                        className='dropdown',
                        placeholder="Select population growth rate..."
                    )
                ]),
                html.Div(className='input-box', children=[
                    html.H3("🤱 Pregnancy Rate Trend"),
                    dcc.Dropdown(
                        id='pregnancy-growth-dropdown',
                        options=[
                            {'label': '↘️ High Reduction (-15%)', 'value': 'High Reduction'},
                            {'label': '↘️ Medium Reduction (-10%)', 'value': 'Medium Reduction'},
                            {'label': '↘️ Low Reduction (-5%)', 'value': 'Low Reduction'},
                            {'label': '↗️ Low Growth (+1%)', 'value': 'Low Growth'},
                            {'label': '↗️ Medium Growth (+2%)', 'value': 'Medium Growth'},
                            {'label': '↗️ High Growth (+3%)', 'value': 'High Growth'}
                        ],
                        value='Medium Growth',
                        className='dropdown',
                        placeholder="Select pregnancy rate trend..."
                    )
                ]),
                html.Div(className='input-box', children=[
                    html.H3("🤰 Initial Pregnancy Rate"),
                    dcc.Input(
                        id='initial-pregnancy-rate-input',
                        type='number',
                        value=0.02,  # 2% initial pregnancy rate
                        step=0.001,
                        className='input',
                        placeholder="Enter initial pregnancy rate (as decimal)..."
                    )
                ])
            ]),
            # Sliders
            html.H2("Settings", className='section-title hide-when-collapsed'),
            html.Div(className='slider-box hide-when-collapsed', children=[
                html.H3("🛡️ Adaptation Effectiveness"),
                dcc.Slider(
                    id='adaptation-effectiveness-slider',
                    min=0,
                    max=100,
                    step=1,
                    value=50,
                    marks={i: f'{i}%' for i in range(0, 101, 25)},
                    className='slider',
                    tooltip={"placement": "bottom", "always_visible": True}
                )
            ]),
            html.Div(className='slider-box hide-when-collapsed', children=[
                html.H3("📅 Analysis Time Period"),
                dcc.RangeSlider(
                    id='time-period-slider',
                    min=2024,
                    max=2050,
                    step=1,
                    marks={year: str(year) for year in range(2024, 2051, 4)},
                    value=[2024, 2030],
                    className='slider',
                    tooltip={"placement": "bottom", "always_visible": True}
                )
            ])
        ]),

        # Main Content
        html.Div(className='main', children=[
            # Featured Indicators Row
            html.Div(className='graphs-section', children=[
                html.H2("Key Indicators", className='section-title'),
                html.Div(className='featured-indicators', children=[
                    html.Div(className='indicator-card', children=[
                        html.H3("Additional Cases"),
                        html.Div(id='total-additional-outcomes-display', className='value')
                    ]),
                    html.Div(className='indicator-card', children=[
                        html.H3("Total Cost"),
                        html.Div(id='total-cost-display', className='value')
                    ]),
                    html.Div(className='indicator-card', children=[
                        html.H3("Mitigation Savings"),
                        html.Div(id='total-mitigation-savings-display', className='value')
                    ]),
                    html.Div(className='indicator-card', children=[
                        html.H3("Adaptation Savings"),
                        html.Div(id='total-adaptation-savings-display', className='value')
                    ])
                ])
            ]),

            # Secondary Indicators
            html.Div(className='outputs-section', children=[
                html.Div(className='secondary-indicators', children=[
                    html.Div(className='key-indicator', children=[
                        html.H2("Odds Ratio"),
                        html.P(id='odds-and-rate-display')
                    ]),
                    html.Div(className='key-indicator', children=[
                        html.H2("Current Rate"),
                        html.P(id='current-rate-display')
                    ]),
                    html.Div(className='key-indicator', children=[
                        html.H2("Pregnancy Rate"),
                        html.P(id='initial-pregnancy-rate-display')
                    ]),
                    html.Div(className='key-indicator', children=[
                        html.H2("Adaptation Effect"),
                        html.P(id='adaptation-effectiveness-display')
                    ]),
                    html.Div(className='key-indicator', children=[
                        html.H2("Cost per Case"),
                        html.P(id='cost-per-case-display')
                    ]),
                    html.Div(className='key-indicator', children=[
                        html.H2("Attributable Fraction"),
                        html.P(id='attributable-fraction-display')
                    ]),
                    html.Div(className='key-indicator', children=[
                        html.H2("Cases Prevented (Mitigation)"),
                        html.P(id='total-mitigation-prevented-display')
                    ]),
                    html.Div(className='key-indicator', children=[
                        html.H2("Cases Prevented (Adaptation)"),
                        html.P(id='total-adaptation-prevented-display')
                    ])
                ])
            ]),

            # Graphs
            html.Div(className='graphs-section', children=[
                html.H2("Projections & Analysis", className='section-title'),
                html.Div(className='graph-container', children=[
                    dcc.Graph(id='outcome-projection-graph', config={'displayModeBar': True, 'displaylogo': False}),
                    dcc.Graph(id='additional-outcomes-prevented-graph', config={'displayModeBar': True, 'displaylogo': False}),
                    dcc.Graph(id='cost-projection-graph', config={'displayModeBar': True, 'displaylogo': False}),
                    dcc.Graph(id='cost-savings-graph', config={'displayModeBar': True, 'displaylogo': False})
                ])
            ]),

            # Export
            html.Div(className='export-section', children=[
                html.Button("📊 Export Data", id="export-button", className='button'),
                dcc.Download(id="download-dataframe-csv")
            ])
        ])
    ])
])

# Step 4: Add Interactivity

# Sidebar collapse state store
store = dcc.Store(id='sidebar-state', data={'collapsed': False})

@app.callback(
    Output('sidebar', 'className'),
    Input('sidebar-toggle', 'n_clicks'),
    State('sidebar', 'className'),
    prevent_initial_call=True
)
def toggle_sidebar(n_clicks, current_class):
    if 'collapsed' in current_class:
        return 'sidebar expanded'
    return 'sidebar collapsed'

@app.callback(
    Output('current-population-input', 'value'),
    Output('current-population-input', 'disabled'),
    Input('country-dropdown', 'value'),
    Input('manual-input-checklist', 'value')
)
def update_population_input(country, manual_input):
    if 'manual' in manual_input:
        return dash.no_update, False
    if country == 'global':
        return 7000000000, True
    population = fetch_population_data(country)
    if population is None:
        population = 7000000000
    return population, True

@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('current-rate-display', 'children'),
    Output('initial-pregnancy-rate-display', 'children'),
    Output('adaptation-effectiveness-display', 'children'),
    Output('cost-per-case-display', 'children'),
    Output('attributable-fraction-display', 'children'),
    Output('total-additional-outcomes-display', 'children'),
    Output('total-cost-display', 'children'),
    Output('total-mitigation-savings-display', 'children'),
    Output('total-adaptation-savings-display', 'children'),
    Output('total-mitigation-prevented-display', 'children'),
    Output('total-adaptation-prevented-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcomes-prevented-graph', 'figure'),
    Output('cost-projection-graph', 'figure'),
    Output('cost-savings-graph', 'figure'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value'),
    Input('country-dropdown', 'value'),
    Input('manual-input-checklist', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness, country, manual_input):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    
    # Fetch population data if manual input is not selected
    if 'manual' not in manual_input:
        if country == 'global':
            current_population = 7000000000
        else:
            current_population = fetch_population_data(country)
            if current_population is None:
                current_population = 7000000000

    # Calculate Attributable Fraction
    attributable_fraction = calculate_attributable_fraction(odds_ratio)
    
    # Calculate selected scenario outcomes and costs
    selected_years, effective_additional_outcomes, costs, additional_outcomes, effective_additional_outcomes_lower, effective_additional_outcomes_upper, costs_lower, costs_upper = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    # Calculate worst scenario (RCP 8.5, 0% adaptation effectiveness) outcomes and costs
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate 0% adaptation scenario outcomes and costs for comparison
    _, zero_adaptation_outcomes, zero_adaptation_costs, _, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate cost savings
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    
    # Calculate additional outcomes prevented
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]
    
    total_additional_outcomes = sum(effective_additional_outcomes)
    total_cost = sum(costs)
    total_mitigation_savings = sum(mitigation_savings)
    total_adaptation_savings = sum(adaptation_savings)
    total_mitigation_prevented = sum(mitigation_prevented)
    total_adaptation_prevented = sum(adaptation_prevented)
    
    display_text = f"{mnch_outcomes[outcome]['odds_ratio']} (95% CI: {mnch_outcomes[outcome]['ci_lower']} - {mnch_outcomes[outcome]['ci_upper']})"
    current_rate_text = f"{mnch_outcomes[outcome]['current_rate']*100}%"
    initial_pregnancy_rate_text = f"{initial_pregnancy_rate*100}%" #convert to percentage
    adaptation_effectiveness_text = f"{adaptation_effectiveness}%"
    cost_per_case_text = f"${mnch_outcomes[outcome]['cost_per_case']:,}"
    attributable_fraction_text = f"{round(attributable_fraction * 100, 2)}%"
    total_additional_outcomes_text = f"{int(round(total_additional_outcomes)):,}"
    total_cost_text = f"${round(total_cost, 2):,}"
    total_mitigation_savings_text = f"${round(total_mitigation_savings, 2):,}"
    total_adaptation_savings_text = f"${round(total_adaptation_savings, 2):,}"
    total_mitigation_prevented_text = f"{int(round(total_mitigation_prevented)):,}"
    total_adaptation_prevented_text = f"{int(round(total_adaptation_prevented)):,}"
    
    # Enhanced graph styling
    graph_config = {
        'font': {'family': '-apple-system, BlinkMacSystemFont, SF Pro Display, sans-serif', 'size': 14},
        'plot_bgcolor': '#FAFAFA',
        'paper_bgcolor': '#FFFFFF',
        'colorway': ['#007AFF', '#FF3B30', '#30D158', '#FF9500', '#AF52DE', '#FF2D92', '#5856D6', '#32D74B']
    }
    
    fig_outcomes = go.Figure()
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines+markers', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='#007AFF', width=3), marker=dict(size=6, color='#007AFF')))
    fig_outcomes.add_trace(go.Scatter(x=selected_years + selected_years[::-1], y=effective_additional_outcomes_upper + effective_additional_outcomes_lower[::-1], fill='toself', fillcolor='rgba(0, 122, 255, 0.1)', line=dict(color='rgba(255, 255, 255, 0)'), showlegend=False, name='Confidence Interval'))
    temperature_increase_per_year = calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1])
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines+markers', name='Temperature Increase', yaxis='y2', line=dict(color='#FF3B30', width=3), marker=dict(size=6, color='#FF3B30')))
    fig_outcomes.update_layout(title=dict(text=f"Climate Impact on {outcome} Cases ({time_period[0]}-{time_period[1]})", font=dict(size=18, family='-apple-system, BlinkMacSystemFont, SF Pro Display, sans-serif'), x=0.5), xaxis_title="Year", yaxis=dict(title="Additional Health Cases", tickfont=dict(color="#007AFF")), yaxis2=dict(title="Temperature Increase (°C)", tickfont=dict(color="#FF3B30"), overlaying='y', side='right'), **graph_config, legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01, bgcolor='rgba(255,255,255,0.8)'))
    
    fig_additional_outcomes_prevented = go.Figure()
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=worst_additional_outcomes, mode='lines+markers', name='Worst Case (RCP 8.5, 0% Adaptation)', line=dict(color='#FF3B30', width=3), marker=dict(size=6, color='#FF3B30')))
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines+markers', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='#007AFF', width=3), marker=dict(size=6, color='#007AFF')))
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years + selected_years[::-1], y=worst_additional_outcomes + effective_additional_outcomes[::-1], fill='toself', fillcolor='rgba(48, 209, 88, 0.2)', line=dict(color='rgba(255, 255, 255, 0)'), showlegend=True, name='Mitigation Benefit'))
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years + selected_years[::-1], y=zero_adaptation_outcomes + effective_additional_outcomes[::-1], fill='toself', fillcolor='rgba(0, 122, 255, 0.2)', line=dict(color='rgba(255, 255, 255, 0)'), showlegend=True, name='Adaptation Benefit'))
    fig_additional_outcomes_prevented.update_layout(title=dict(text=f"Prevention Impact: Cases Avoided ({time_period[0]}-{time_period[1]})", font=dict(size=18, family='-apple-system, BlinkMacSystemFont, SF Pro Display, sans-serif'), x=0.5), xaxis_title="Year", yaxis_title="Additional Health Cases", **graph_config, legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01, bgcolor='rgba(255,255,255,0.8)'))
    
    fig_costs = go.Figure()
    fig_costs.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines+markers', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='#007AFF', width=3), marker=dict(size=6, color='#007AFF')))
    fig_costs.add_trace(go.Scatter(x=selected_years + selected_years[::-1], y=costs_upper + costs_lower[::-1], fill='toself', fillcolor='rgba(0, 122, 255, 0.1)', line=dict(color='rgba(255, 255, 255, 0)'), showlegend=False, name='Cost Range'))
    fig_costs.update_layout(title=dict(text=f"Economic Impact: Healthcare Costs ({time_period[0]}-{time_period[1]})", font=dict(size=18, family='-apple-system, BlinkMacSystemFont, SF Pro Display, sans-serif'), x=0.5), xaxis_title="Year", yaxis_title="Healthcare Costs (USD)", **graph_config, legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01, bgcolor='rgba(255,255,255,0.8)'))
    
    fig_cost_savings = go.Figure()
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=worst_costs, mode='lines+markers', name='Worst Case (RCP 8.5, 0% Adaptation)', line=dict(color='#FF3B30', width=3), marker=dict(size=6, color='#FF3B30')))
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines+markers', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='#007AFF', width=3), marker=dict(size=6, color='#007AFF')))
    fig_cost_savings.add_trace(go.Scatter(x=selected_years + selected_years[::-1], y=worst_costs + costs[::-1], fill='toself', fillcolor='rgba(48, 209, 88, 0.2)', line=dict(color='rgba(255, 255, 255, 0)'), showlegend=True, name='Mitigation Savings'))
    fig_cost_savings.add_trace(go.Scatter(x=selected_years + selected_years[::-1], y=zero_adaptation_costs + costs[::-1], fill='toself', fillcolor='rgba(0, 122, 255, 0.2)', line=dict(color='rgba(255, 255, 255, 0)'), showlegend=True, name='Adaptation Savings'))
    fig_cost_savings.update_layout(title=dict(text=f"Economic Benefits: Cost Savings Analysis ({time_period[0]}-{time_period[1]})", font=dict(size=18, family='-apple-system, BlinkMacSystemFont, SF Pro Display, sans-serif'), x=0.5), xaxis_title="Year", yaxis_title="Healthcare Costs (USD)", **graph_config, legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01, bgcolor='rgba(255,255,255,0.8)'))
    
    return (
        display_text,
        current_rate_text,
        initial_pregnancy_rate_text,
        adaptation_effectiveness_text,
        cost_per_case_text,
        attributable_fraction_text,
        total_additional_outcomes_text,
        total_cost_text,
        total_mitigation_savings_text,
        total_adaptation_savings_text,
        total_mitigation_prevented_text,
        total_adaptation_prevented_text,
        fig_outcomes,
        fig_additional_outcomes_prevented,
        fig_costs,
        fig_cost_savings
    )

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    selected_years, effective_additional_outcomes, costs, additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    _, zero_adaptation_outcomes, zero_adaptation_costs, _, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1]),
        'Costs': costs,
        'Worst Case Costs': worst_costs,
        'Zero Adaptation Costs': zero_adaptation_costs,
        'Mitigation Savings': mitigation_savings,
        'Adaptation Savings': adaptation_savings,
        'Mitigation Prevented': mitigation_prevented,
        'Adaptation Prevented': adaptation_prevented
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "climate_health_impact_analysis.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run(debug=True)


Fetching population data for ZAF...
Fetching population data for ZAF...
Fetching population data for ZAF...
Fetching population data for ZAF...
Fetching population data for ZAF...
Fetching population data for ZAF...
Fetching population data for ZAF...
